In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

## Datos
data = pd.read_csv("data.csv")
print("El numero de datos en el csv es:",len(data))
data.head()
x = data["x"].values
y = data["y"].values
t = data["t"].values
N=len(x)

## Vista a la evolucion de los datos X y Y en el tiempo.
figure = plt.figure(figsize=(6.4,6.0),constrained_layout=True)
gs0 = figure.add_gridspec(nrows=2, ncols=1)
ax1 = figure.add_subplot(gs0[0,0:])
ax1.set_title("Dispercion de variable X en t")
ax1.plot(t,x,".",label="Experimental")
ax1.grid(color="#F5BCA9")
ax1.set_ylabel("x")
ax1.legend(loc='lower left', fontsize='small')

ax2 = figure.add_subplot(gs0[1,0:])
ax2.set_title("Dispercion de variable Y en t")
ax2.plot(t,y,".",label="Experimental")
ax2.grid(color="#F5BCA9")
ax2.set_ylabel("y")
ax2.legend(loc='lower left', fontsize='small')

plt.show()

## Modelos predictivos
def f(t,a,b,theta1,theta2,c,w1,w2): ##Usando popt12(100)
    return a*np.sin(w1*np.pi*t+theta1)+b*np.sin(w2*np.pi*t+theta2)+c
def g(t,a,theta,b,w): ##Usando popt12(200)
    return a*np.sin(w*np.pi*t+theta)+b
def z(t,a,b,theta1,theta2,c,w1,w2): ##Usando popt12(100)
    return a*np.cos(w1*2*np.pi*t+theta1)+b*np.sin(w2*np.pi*t+theta2)+c ##Usando popt12(100)
def comb(g,a1,b1,c1,a2,b2,theta1,theta2,theta3,w1,w2,w3):
    tt = (1/(w3*np.pi))*(np.arcsin((g-b2)/a2) - theta3)
    return a1*np.sin(w1*np.pi*tt+theta1)+b1*np.sin(w2*np.pi*tt+theta2)+c1

## Definicion de Chi cuadrado
def chisquar(data_obs,eq_fit):
    solv_fit = np.sum(np.abs(((data_obs - eq_fit)**2) / data_obs))
    return solv_fit

## Minimo de chi cuadrado para 7 parametros
def minimo_chisquare7(t,a,b,c,theta1,theta2,w1,w2,eq_fit,data_obs):
    for i in a:
        for j in b:
            for k in c:
                for s in theta:
                    for l in theta2:
                        for v in w1:
                            for r in w2:
                                if i == a[0]:
                                    acumul = chisquar(data_obs, eq_fit(t,i,j,k,s,l,v,r))
                                    parameters = [i,j,k,s,l,v,r]
                                else:
                                    if acumul > chisquar(data_obs,eq_fit(t,i,j,k,s,l,v,r)):
                                        acumul = chisquar(data_obs,eq_fit(t,i,j,k,s,l,v,r))
                                        parameters = [i,j,k,s,l],v,r
    return acumul , parameters

## Minimo de chi cuadrado para 4 parametros
def minimo_chisquare4(t,a,b,theta,w,eq_fit,data_obs):
    for i in a:
        for j in b:
            for k in theta:
                for s in w:
                    if i == a[0]:
                        acumul = chisquar(data_obs, eq_fit(t,i,j,k,s))
                        parameters = [i,j,k,s]
                    else:
                        if acumul > chisquar(data_obs,eq_fit(t,i,j,k,s)):
                            acumul = chisquar(data_obs,eq_fit(t,i,j,k,s))
                            parameters = [i,j,k,s]
    return acumul , parameters

## Se define los rangos para los parametros
a = np.arange(0,1,0.03)
b = np.arange(1,2,0.03)
c = np.arange(-2,1,0.02)
thetha1 = np.arange(-10,-5,0.03)
thetha2 = np.arange(0,2,0.03)
w1 =  np.arange(0,100,0.53)
w2 = np.arange(-20,10,0.53)

## Muestra la solucion 
minin_chi1,dat_x1 = minimo_chisquare7(t,a,b,c,theta1,theta2,w1,w2,z,x) # completo
minin_chi2,dat_x2 = minimo_chisquare7(t[:100],a,b,c,theta1,theta2,w1,w2,z,x[:100]) # intervalo 


## Solucion de cada ajuste y suma de ajustes.
fig = plt.figure(constrained_layout=True)
gs1 = fig.add_gridspec(nrows=3, ncols=6)
f_ax1 = fig.add_subplot(gs1[0,:2])
f_ax1.plot(t,z(t, *dat_x1))
f_ax1.grid(color="#F5BCA9")
f_ax1.set_title("Primer ajuste")
f_ax1.set_ylabel("x")


f_ax2 = fig.add_subplot(gs1[1, :2])
f_ax2.plot(t,z(t, *dat_x2)+1)
f_ax2.grid(color="#F5BCA9")
f_ax2.set_title("Segundo ajuste")
f_ax2.set_ylabel("x")

f_ax3 = fig.add_subplot(gs1[2, :2])
f_ax3.plot(t,z(t, *dat_x1)+z(t, *dat_x2)+1)
f_ax3.grid(color="#F5BCA9")
f_ax3.set_title("Combinacion lineal")
f_ax3.set_ylabel("x")
f_ax3.set_xlabel("t")

f_ax4 = fig.add_subplot(gs1[:3,-4:])
f_ax4.plot(t,x,label="Experimental")
f_ax4.plot(t,z(t, *dat_x1)+z(t, *dat_x2)+1,label="Teorico")
f_ax4.grid(color="#F5BCA9")
f_ax4.set_title("Comparacion experimental con teorico")
f_ax4.set_xlabel("t")
f_ax4.legend(loc='lower left', fontsize='small')

plt.show()

## Valor de chi cuadrado:
fun = z(t, *dat_x1)+z(t, *dat_x2)+1
chi = chisquar(x,fun)
print(chi)

## Matriz de covarianza de pearson:
print(data.corr(method="pearson"))

## Resultado ajuste covarianza
minin_chi3,dat_x3 = minimo_chisquare4(t,a,b,theta1,w,g,x) # completo
minin_chi4,dat_x4 = minimo_chisquare4(t[:200],a,b,theta1,w,g,x[:200]) # intervalo 

minin_chi5,dat_x5 = minimo_chisquare7(t,a,b,c,theta1,theta2,w1,w2,f,y) # completo
minin_chi6,dat_x6 = minimo_chisquare7(t[:400],a,b,c,theta1,theta2,w1,w2,f,y[:400]) # intervalo 

fun2 = g(t, *dat_x3)+g(t, *dat_x4)+1
fun3 = f(t, *dat_x5)+f(t, *dat_x6)+1

plt.plot(y,x,".",label="Datos experimentales")
plt.grid(color="#F5BCA9")
plt.xlabel("y")
plt.ylabel("x")
plt.plot(fun3,fun2,".",label="Mejores ajustes de las funciones x,y")
plt.title("Dispersion de la variable X respecto a Y")
plt.legend()
plt.show()